In [5]:
import pandas as pd
import numpy as np
import glob as glob
import math as math
import scipy.stats as stats

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
#import seaborn.apionly as sns
import yaml

#import seaborn as sns
#sns.set(color_codes = True)

from matplotlib import gridspec
from matplotlib import colors
from matplotlib import artist
from matplotlib import rc_context

#from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import percentileofscore

%matplotlib inline

In [6]:
# look up the rescaled copy number from a segDF given a chromosome and position
def LookupCN(row, segDF): 
    # look up the segment containing the chromosome and position
    chrom = row['Chromosome']
    pos = row['Start_position'] + (row['End_position'] - row['Start_position'])/2 #midpoint
    crit = segDF['Chromosome'].astype(str) == str(chrom)
    startBP = segDF[crit]['Start.bp']
    endBP = segDF[crit]['End.bp']
    segCrit = pd.Series([int(pos) >= int(s) for s in startBP]) & pd.Series([int(pos) <= int(s) for s in endBP])
#    print segDF[crit][list(segCrit)]
    
    minorCN = segDF[crit][list(segCrit)]['rescaled.cn.a1'] # should only be one
    majorCN = segDF[crit][list(segCrit)]['rescaled.cn.a2'] # should only be one
    totalCN = segDF[crit][list(segCrit)]['rescaled_total_cn'] # should only be one

    # round to the nearest integer if not already an integer -- consistent with the copy number 
    # of the largest clone
    if minorCN.isnull().all(): # occurs when we have insufficient hets for the segment to generate allelic copy numbers
        if totalCN.isnull().all(): # occurs when this position falls inbetween segments, default baseline 1:1
            minorCN = 1
            majorCN = 1
        else: # insufficient hets to call allelic differences
            totalCN = int(round(totalCN))
            if totalCN > 1:
                minorCN = 1
                majorCN = totalCN - 1
            else: # totalCN is 0 or 1
                minorCN = 0
                majorCN = totalCN
#    print chrom, pos, minorCN, majorCN

    minorCN = int(round(minorCN))
    majorCN = int(round(majorCN))

    return minorCN, majorCN
    

# GeneratePycloneTumor - given mutational data and the post-purity corrected copy numbers from a
# seg file, returns a DF in the format expected by Pyclone
def GeneratePycloneTumor(mutDF, segDF):
    columns = ['mutation_id', 'ref_counts', 'var_counts', 'normal_cn', 'minor_cn', 'major_cn', 
               'sample_id', 'ccf_hat', 'purity']
    outDF = pd.DataFrame()
    outDF['mutation_id'] = mutDF['Hugo_Symbol'] + ":chr" + mutDF['Chromosome'].astype(str) + "_" + mutDF['Start_position'].astype(str) + "_" + mutDF['End_position'].astype(str)
    outDF['sample_id'] = mutDF['sample']
    outDF['ref_counts'] = mutDF['ref']
    outDF['alt_counts'] = mutDF['alt']
    outDF['normal_cn'] = [2] * len(mutDF.index)
    outDF['ccf_hat'] = mutDF['ccf_hat']
    outDF['tumour_content'] = mutDF['purity']
    # generate copy numbers
    output = mutDF.apply(LookupCN, axis=1, args=(segDF,))
    minorCN,majorCN = zip(*output)
    outDF['minor_cn'] = minorCN
    outDF['major_cn'] = majorCN
    # Filter for null mutations with missing ref/alt counts (these come from forcecall)
    outDF = outDF[~outDF['ref_counts'].isnull()]

    # Some ref/alt counts have multiple values (e.g. DNPs) that look like "xx|xx"
    # take the first value
    outDF['ref_counts'] = [str(s).split("|")[0] for s in outDF['ref_counts']]
    outDF['alt_counts'] = [str(s).split("|")[0] for s in outDF['alt_counts']]
    
    # remove any where majorCN == 0 # can occur with homozygous deletions
    removeCrit = outDF['major_cn']==0
    outDF = outDF[~removeCrit]
    return outDF

# Input all the post-absolute purity and ploidy corrected mutational and segment files
mutFiles = "../WES/2_SNV/Absolute_merged/*ABS_MAF.txt"
segFiles = "../WES/3_CNA/segtab/*segtab.txt"
mutFiles = glob.glob(mutFiles)
segFiles = glob.glob(segFiles)

outDir = "../WES/4_PyClone/"

tumors = []

for mutFile in mutFiles:
    tumorId = mutFile.split("/")[-1].split("_ABS_MAF.txt")[0]
    print "Processing", tumorId,"..."
    segFile = [s for s in segFiles if tumorId in s]
    if len(segFile) != 1:
        print "Either not found or more than one segfile matching the tumorId", tumorId
        print segFile
    else: # just one found, go ahead and process
        segFile = segFile[0]
        if True:
            mutDF = pd.read_csv(mutFile, sep="\t", low_memory=False)
            segDF = pd.read_csv(segFile, sep="\t", low_memory=False)
            pycloneDF = GeneratePycloneTumor(mutDF, segDF)
            pycloneDF.to_csv(outDir + tumorId + ".pyclone.tsv", sep = "\t", index=False)
        tumors += [tumorId]




Processing Patient136_N-T19 ...
Processing Patient136_N-T10 ...
Processing Patient136_N-T15 ...
Processing Patient136_N-T1 ...
Either not found or more than one segfile matching the tumorId Patient136_N-T1
['../WES/3_CNA/segtab/Patient136_N-T13.segtab.txt', '../WES/3_CNA/segtab/Patient136_N-T16.segtab.txt', '../WES/3_CNA/segtab/Patient136_N-T11.segtab.txt', '../WES/3_CNA/segtab/Patient136_N-T14.segtab.txt', '../WES/3_CNA/segtab/Patient136_N-T1.segtab.txt', '../WES/3_CNA/segtab/Patient136_N-T19.segtab.txt', '../WES/3_CNA/segtab/Patient136_N-T17.segtab.txt', '../WES/3_CNA/segtab/Patient136_N-T15.segtab.txt', '../WES/3_CNA/segtab/Patient136_N-T18.segtab.txt', '../WES/3_CNA/segtab/Patient136_N-T10.segtab.txt']
Processing Patient136_N-T8 ...
Processing Patient136_N-T16 ...
Processing Patient136_N-T2 ...
Either not found or more than one segfile matching the tumorId Patient136_N-T2
['../WES/3_CNA/segtab/Patient136_N-T21.segtab.txt', '../WES/3_CNA/segtab/Patient136_N-T2.segtab.txt']
Processin

In [12]:
all_files = glob.glob("../WES/4_PyClone/*.tsv")

df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

concatenated_df.to_csv("../WES/4_PyClone/merged_pyclone.tsv", sep = "\t", index=False, doublequote=False)

In [ ]:
concatenated_df.to_csv(sep = "\t", index=False, doublequote=False)